In [2]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


from enum import Enum
from pydantic import BaseModel, Field

In [3]:
import getpass
import os
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")


In [4]:
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

In [5]:
class BookingCarDetails(BaseModel):
    
    """Details for the bookings car details"""
    name: str = Field(
        ...,
        description="The name of the person booking the ride.This is optional if provided",
    )
    number_phone: str = Field(
        ...,
        description="The phone number of the user",
    )
    pick_up_location: str = Field(
        ...,
        description="The location where the user will be picked up. This can be a full address or a specific location name.",
    )
    destination_location: str = Field(
        ...,
        description="The destination location for the ride. This can be a full address or a specific location name."
    )
    pick_up_time: str = Field(
        ...,
        description="The time the user intends to be picked up. No format keeps the text related to time."
    )

In [9]:
chain = llm.with_structured_output(BookingCarDetails)

In [10]:
reponse = chain.invoke("i want to book a car to 271 Nguyen Van Linh, Da Nang from 460 Tran Dai Nghia, Da Nang at 9 tomorrow ")

In [11]:
reponse

BookingCarDetails(name='', number_phone='', pick_up_location='460 Tran Dai Nghia, Da Nang', destination_location='271 Nguyen Van Linh, Da Nang', pick_up_time='9 tomorrow')

In [6]:
def check_what_is_empty(user_peronal_details):
    ask_for = []
    # Check if fields are empty
    for field, value in user_peronal_details.model_dump().items():
        if value in [None, "", 0]:  # You can add other 'empty' conditions as per your requirements
            print(f"Field '{field}' is empty.")
            ask_for.append(f'{field}')
    return ask_for

In [12]:
ask_for = check_what_is_empty(reponse)
ask_for

Field 'name' is empty.
Field 'number_phone' is empty.


['name', 'number_phone']

In [13]:
def add_non_empty_details(current_details: BookingCarDetails, new_details: BookingCarDetails):
    non_empty_details = {k: v for k, v in new_details.model_dump().items() if v not in [None, ""]}
    updated_details = current_details.model_copy(update=non_empty_details)
    return updated_details

In [187]:
booking_details = BookingCarDetails(name="",number_phone="",pick_up_location="",destination_location="", pick_up_time="")

In [22]:
booking_details = add_non_empty_details(booking_details ,reponse)

NameError: name 'booking_details' is not defined

In [189]:
booking_details

BookingCarDetails(name='', number_phone='', pick_up_location='460 Tran Dai Nghia, Da Nang', destination_location='271 Nguyen Van Linh, Da Nang', pick_up_time='9 tomorrow')

In [204]:
ask_for = check_what_is_empty(booking_details)
ask_for

Field 'name' is empty.
Field 'number_phone' is empty.


['name', 'number_phone']

In [16]:
def ask_for_info(ask_list:list):
    # prompt template 1
    first_prompt = ChatPromptTemplate.from_template(
        "Below is are some things to ask the user for in a coversation way. you should only ask one question at a time even if you don't get all the info \
        don't ask as a list! Don't greet the user! Don't say Hi.Explain you need to get some info. If the ask_for list is empty then thank them and ask how you can help them \n\n \
        ### ask_for list: {ask_for}"
    )
    # info_gathering_chain
    info_gathering_chain = first_prompt | llm | StrOutputParser()
    ai_chat = info_gathering_chain.invoke({"ask_for": ask_list})
    print(first_prompt)
    return ai_chat

In [17]:
def filter_response(text_input, user_details ):
    chain = llm.with_structured_output(BookingCarDetails)
    res = chain.invoke(text_input)
    # add filtered info to the
    user_details = add_non_empty_details(user_details,res)
    print(user_details)
    ask_for = check_what_is_empty(user_details)
    return user_details, ask_for

In [23]:
booking_details

NameError: name 'booking_details' is not defined

In [14]:
ask_for

['name', 'number_phone']

In [18]:
ask_for_info(ask_for)

input_variables=['ask_for'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ask_for'], input_types={}, partial_variables={}, template="Below is are some things to ask the user for in a coversation way. you should only ask one question at a time even if you don't get all the info         don't ask as a list! Don't greet the user! Don't say Hi.Explain you need to get some info. If the ask_for list is empty then thank them and ask how you can help them \n\n         ### ask_for list: {ask_for}"), additional_kwargs={})]


'I need to gather some information from you. Could you please tell me your name?'

In [19]:
text_input ="ok My name is Sam"

In [20]:
user_details, ask_for = filter_response(text_input, booking_details)

NameError: name 'booking_details' is not defined

In [218]:
user_details

BookingCarDetails(name='', number_phone='0917181880', pick_up_location='460 Tran Dai Nghia, Da Nang', destination_location='271 Nguyen Van Linh, Da Nang', pick_up_time='9 tomorrow')

In [152]:
if ask_for:
    ai_response = ask_for_info(ask_for)
    print(ai_response)
else:
    print('Everything gathered move to next phase')

input_variables=['ask_for'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ask_for'], input_types={}, partial_variables={}, template="Below is are some things to ask the user for in a coversation way. you should only ask one question at a time even if you don't get all the info         don't ask as a list! Don't greet the user! Don't say Hi.Explain you need to get some info. If the ask_for list is empty then thank them and ask how you can help them \n\n         ### ask_for list: {ask_for}"), additional_kwargs={})]
I need to gather some information from you. Could you please provide your phone number?


In [219]:
text_input ="ok My name is Sam"
user_details, ask_for = filter_response(text_input, user_details)

name='Sam' number_phone='0917181880' pick_up_location='460 Tran Dai Nghia, Da Nang' destination_location='271 Nguyen Van Linh, Da Nang' pick_up_time='9 tomorrow'


In [220]:
if ask_for:
    ai_response = ask_for_info(ask_for)
    print(ai_response)
else:
    print('Everything gathered move to next phase')

Everything gathered move to next phase


In [128]:
if ask_for:
    ai_response = ask_for_info(ask_for)
    print(ai_response)
else:
    print('Everything gathered move to next phase')

input_variables=['ask_for'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ask_for'], input_types={}, partial_variables={}, template="Below is are some things to ask the user for in a coversation way. you should only ask one question at a time even if you don't get all the info         don't ask as a list! Don't greet the user! Don't say Hi.Explain you need to get some info. If the ask_for list is empty then thank them and ask how you can help them \n\n         ### ask_for list: {ask_for}"), additional_kwargs={})]
I need to gather some information from you. Could you please tell me your name?


In [2]:
import requests
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field 
from langchain_core.tools import tool
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

class BookingCarDetails(BaseModel):
    """Details for the bookings car details"""
    name: str = Field(
        ...,
        description="The name of the person booking the ride.This is optional if provided",
    )
    number_phone: str = Field(
        ...,
        description="The phone number of the user.This is optional if provided",
    )
    pick_up_location: str = Field(
        ...,
        description="The location where the user will be picked up. This can be a full address or a specific location name.This is optional if provided",
    )
    destination_location: str = Field(
        ...,
        description="The destination location for the ride. This can be a full address or a specific location name.This is optional if provided"
    )
    pick_up_time: str = Field(
        ...,
        description="The time the user intends to be picked up. No format keeps the text related to time..This is optional if provided"
    )
    # @feild_validator('')


def check_what_is_empty(user_personal_details):
    ask_for = []
    # Check if fields are empty
    for field, value in user_personal_details.model_dump().items():
        if value in [None, "", 0]:  # Add other 'empty' conditions if needed
            print(f"Field '{field}' is empty.")
            ask_for.append(field)
    return ask_for


def add_non_empty_details(current_details: BookingCarDetails, new_details: BookingCarDetails):
    non_empty_details = {k: v for k, v in new_details.model_dump().items() if v not in [None, ""]}
    updated_details = current_details.model_copy(update=non_empty_details)
    return updated_details


def ask_for_info(ask_list: list):
    first_prompt = ChatPromptTemplate.from_template(
        """Ask one question at a time, even if you don't get all the info. Don't list the questions or greet the user. 
        Explain you're gathering info to help. If 'ask_for' is empty, thank the user and ask how you can assist next.
        ### ask_for list: {ask_for}"""
    )

    info_gathering_chain = first_prompt | llm | StrOutputParser()
    ai_chat = info_gathering_chain.invoke({"ask_for": ask_list})
    print(first_prompt)
    return ai_chat
def filter_response(text_input, user_details ):
    chain = llm.with_structured_output(BookingCarDetails)
    res = chain.invoke(text_input)
    # add filtered info to the
    user_details = add_non_empty_details(user_details,res)
    print(user_details)
    ask_for = check_what_is_empty(user_details)
    return user_details, ask_for
def ask_confirm_info(booking_details: BookingCarDetails):
    # booking_details.
    message = (
        f"Please confirm your ride details:\n"
        f"- Pickup Location: {booking_details.pick_up_location}\n"
        f"- Destination: {booking_details.destination_location}\n"
        f"- Pickup Time: {booking_details.pick_up_time}\n"
        f"- Name: {booking_details.name}\n"
        f"- Contact Number: {booking_details.number_phone}\n"
    )
    print(message)
@tool
def get_booking_details(input_text):
    """ This is function to get information for booking"""
    chain = llm.with_structured_output(BookingCarDetails)
    response_text = "i want to book a car to 271 Nguyen Van Linh, Da Nang from 460 Tran Dai Nghia, Da Nang at 9 tomorrow "
    response = chain.invoke(input_text)
    booking_details = BookingCarDetails(
        name="", number_phone="", pick_up_location="", destination_location="", pick_up_time=""
    )
    booking_details = add_non_empty_details(booking_details, response)
    
    ask_for = check_what_is_empty(booking_details)
    
    ai_response = ask_for_info(ask_for)
    print(ai_response)
    text_input = input()
    user_details, ask_for = filter_response(text_input, booking_details)
    while ask_for:  
        ai_response = ask_for_info(ask_for)
        input
        print(ai_response)
        text_input = input()
        user_details, ask_for = filter_response(text_input, user_details)
        print(ask_for)
    
    ask_confirm_info(user_details)
    return 
    
        

    # # Validate locations
    # validation_errors = validate_locations(booking_details)
    # if validation_errors:
    #     print(f"Validation errors: {validation_errors}")
    # else:
    #     print("All details are valid.")





In [63]:

# chain = llm.with_structured_output(BookingCarDetails)
response = "i want to book a car to 271 Nguyen Van Linh, Da Nang from 460 Tran Dai Nghia, Da Nang at 9 tomorrow "
# response = chain.invoke(response_text)

get_booking_details(response)

Field 'name' is empty.
Field 'number_phone' is empty.
input_variables=['ask_for'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ask_for'], input_types={}, partial_variables={}, template="Ask one question at a time, even if you don't get all the info. Don't list the questions or greet the user. \n        Explain you're gathering info to help. If 'ask_for' is empty, thank the user and ask how you can assist next.\n        ### ask_for list: {ask_for}"), additional_kwargs={})]
I'm gathering some information to help you better. Could you please provide your name?


KeyboardInterrupt: 

In [3]:
tools = [get_booking_details]

In [ ]:
from langgraph.prebuilt import create_react_agent

# system_prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "You are very powerful assistant, but don't know current events",
#         ),
#         ("user", "{input}")
#     ]
# )
system_prompt = """
You are a very powerful assistant. Be polite, clear, and understandable.
If users ask general questions, answer them helpfully. If users want to book a ride, 
call the 'get_booking_details' function to gather booking information, and pass the user's final input to the function.
Remember to keep the user's last input
"""
agent_executor = create_react_agent(llm, tools = [get_booking_details] , state_modifier=system_prompt)

In [ ]:
inputs = {"messages": [("user", "my name is Huy,I want to book a car from 271 Nguyen Van Linh to 466 NGuyen van linh da nang at now")]}
for s in agent_executor.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

my name is Huy, I want to book a car from 271 Nguyen Van Linh to 466 NGuyen van linh da nang at n
================================== Ai Message ==================================
Tool Calls:
  get_booking_details (call_RJfhAWKQCLGoBcChtxgU38to)
 Call ID: call_RJfhAWKQCLGoBcChtxgU38to
  Args:
    input_text: book a car from 271 Nguyen Van Linh to 466 Nguyen Van Linh Da Nang
input_variables=['ask_for'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ask_for'], input_types={}, partial_variables={}, template="Ask one question at a time, even if you don't get all the info. Don't list the questions or greet the user. \n        Explain you're gathering info to help. If 'ask_for' is empty, thank the user and ask how you can assist next.\n        ### ask_for list: {ask_for}"), additional_kwargs={})]
I'm gathering information to help you better. How c